In [43]:
import pandas as pd 
import numpy as np  
from datetime import datetime
from scipy.interpolate import RegularGridInterpolator

### **Case Study 1 - Part 1: Pricing a structured product on Brent crude oil**

 - The aim of this exercise is to compute the current market value, for the bank, (as of 08/04/2022) of a structured product sold to a customer.
The product is sold at 100USD (= Notional N). It has a maturity approx. 3 years (20/11/2024) and pays an annual coupon.

 - Annual coupons are 5% of the notional if the Brent is above 85.50$ per barrel on the coupon date, and 0 otherwise. For each coupon, the reference price is the final settlement price of the futures expiring on the coupon date. Resp. Dec22, Dec23, Dec24 contracts.

 - The redemption at maturity is 100% of the notional N. 

 


___



## ***Solution***

### **Step 1: Import Data (pricing data + volatility matrix)**

In [127]:
# Import the pricing matrix and assign it to a python DataFrame (RAW DATA)


pricing_matrix = pd.read_excel("Case_Study1_Structured_Products_2025.xlsx", 
                               skiprows=28, 
                               nrows=5,
                               sheet_name="Case_Study_Part1",
                               index_col=1)

pricing_matrix["r"] = pd.to_numeric(pricing_matrix["r"], errors="coerce")
pricing_matrix = pricing_matrix.iloc[1:]
pricing_matrix.drop(columns=["Unnamed: 0", "Unnamed: 4"], inplace=True)

# Import Volatility Matrix

t_vol = [0.5, 0.75, 1, 2, 3, 4, 5]

vol_matrix = pd.read_excel("Case_Study1_Structured_Products_2025.xlsx", 
                           skiprows=39,
                            nrows=8,
                            sheet_name="Case_Study_Part1",
                            index_col=2)


vol_matrix.drop(columns=["Unnamed: 0", "Unnamed: 1"], inplace=True)

vol_matrix.index = t_vol

vol_matrix



,80.464,85.493,90.522,95.551,100.58,105.6089,110.6379,115.667,120.696
0.50,49.68,48.24,47.27,46.72,46.52,46.57,46.81,47.18,47.63
0.75,47.26,45.51,44.10,43.10,42.49,42.20,42.13,42.21,42.37
1.00,45.61,43.67,41.99,40.64,39.72,39.29,39.33,39.77,40.48
2.00,40.43,39.13,37.93,36.92,36.32,36.30,36.62,37.04,37.48
3.00,32.66,31.78,31.09,30.42,29.29,27.94,27.06,26.17,25.28
4.00,31.40,30.75,30.74,30.81,29.91,28.57,28.20,27.83,27.47
5.00,30.21,29.64,29.06,28.49,27.13,25.45,24.56,23.66,22.77


In [128]:

# Initiate Start date and create T column

start_date = pd.Timestamp(2022, 4, 8)

pricing_matrix.insert(loc=1, 
                      column="T", 
                      value = (pricing_matrix["Exp. Date"] - start_date).dt.days / 252.0)


# Discount Factor

pricing_matrix["Discount Factor"] = np.exp(-pricing_matrix["r"] * pricing_matrix["T"])



# Digital Option Payoff

K = 85.5

K_vector = np.full_like(pricing_matrix["T"], K) # any assignment of the K vector works
                                                # supposedly the full_like is cleaner

# K_vector = K * np.ones(len(pricing_matrix["T"])) # this occurred to me in a dream


pricing_matrix[r"I {F(0,T) > K}"] = np.where(
    pricing_matrix["F(0,T)"] > K,  
    5 * pricing_matrix["Discount Factor"],  
    0  
)


# Compute value of option of coupon 

# Compute D0




d0 = pd.DataFrame({"D0": np.ones(3)})

d0


,D0
0,1.0
1,1.0
2,1.0


In [ ]:
# Interpolate volatility with a 2D interpolation 
# using K = 85.5 and the the T values in the Pricing Matrix

K_strikes = np.array(vol_matrix.columns, dtype=float)

T_values = vol_matrix.index.values

interp_points = np.column_stack((pricing_matrix["T"].values, K_vector)) # Note: Column_stack expects tuples of 1D arrays

interpol_func = RegularGridInterpolator((T_values, K_strikes), vol_matrix.values)

pricing_matrix["Interpolated Volatility"] = (interpol_func(interp_points)) / 100

pricing_matrix


,Exp. Date,T,"F(0,T)",r,Discount Factor,"I {F(0,T) > K}",Interpolated Volatility
Contracts,,,,,,,
DEC22,2022-11-20,0.896825,96.55,0.0212,0.981167,4.905835,0.444272
DEC23,2023-11-20,2.345238,88.69,0.0274,0.937762,4.688808,0.365911
DEC24,2024-11-20,3.797619,83.52,0.0285,0.897419,0.000000,0.309582
